In [ ]:
##### EKSPERYMENT 3 #################

In [ ]:
### W celu porównania zdolności predykcyjnych oraz preprocessingu frameworków rozważanych w pracy dyplomowej 
### z tradycyjnymi metodami, tworzymy pipeline, który będzie z użyciem algorytmu Random forest przewidywał zmienną celu
### class w zbiorze danych generated3

In [2]:
# importujemy potrzebne rzeczy

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier

In [4]:
# wczytuję potrzebny zbiór 

df = pd.read_csv("generated3.csv")
df

,Unnamed: 0,col1,col2,col3,col4,col5,col6,col7,col8,col9,Class
0,1,112.895254,-1.463421,9.978380,112.895254,16.027942,29.935141,23.286328,-0.731711,No,0
1,2,56.643203,-1.170813,4.451193,56.643203,-49.038639,13.353580,25.323074,-0.585407,Yes,1
2,3,98.492637,0.589826,8.378881,98.492637,-13.451975,25.136642,-18.482716,0.294913,Yes,1
3,4,90.825134,-0.628418,-6.689641,90.825134,16.511973,-20.068924,-12.727652,-0.314209,Yes,0
4,5,126.083134,-2.131950,1.839880,126.083134,-7.126570,5.519639,-1.759253,-1.065975,No,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,98.797112,-4.172046,-9.761022,98.797112,29.434417,-29.283066,-6.550079,-2.086023,No,1
9996,9997,145.288377,-0.560509,0.280363,145.288377,-10.615147,0.841089,6.987163,-0.280254,No,1
9997,9998,82.474812,-1.273415,6.322509,82.474812,-31.068394,18.967528,5.446964,-0.636707,Yes,1
9998,9999,105.075030,-0.713320,1.593137,105.075030,-40.745323,4.779411,-5.748540,-0.356660,Yes,0


In [7]:
y = df.Class
X = df.drop(['Unnamed: 0', 'Class'], axis = 1)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [11]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

categorical_features = list(X.dtypes[(X.dtypes == 'object') | (X.dtypes == 'category')].index)
numerical_features = list(X.dtypes[(X.dtypes != 'object') & (X.dtypes != 'category')].index)

categorical_transformer = Pipeline([
    ('onehot', OneHotEncoder(handle_unknown = 'ignore'))
])

numerical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy = 'mean')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer([
    ('onehot', categorical_transformer, categorical_features),
    ('scaler', numerical_transformer, numerical_features)
])

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline([
    ('pre', preprocessor),
    ('selector', SelectFromModel(LogisticRegression())),
    ('rf', RandomForestClassifier())
])

In [25]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('pre',
                 ColumnTransformer(transformers=[('onehot',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['col9']),
                                                 ('scaler',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['col1', 'col2', 'col3',
                                                   'col4', 'col5', 'col6',
                                                   'col7', 'col8'])])),
                ('selector', SelectFromModel(estimator=Log

In [26]:
pipeline.score(X_test, y_test)

0.5146666666666667

In [ ]:
######## zrobimy jeszcze gridsearch parametrów  ###############

In [33]:
from sklearn.model_selection import GridSearchCV
pipeline = Pipeline([
    ('pre', preprocessor),
    ('selector', SelectFromModel(LogisticRegression())),
    ('rf', RandomForestClassifier())
])

params = {
    
    'rf__max_depth': [5, 6, 7, 8, 9, 10, 11, 12],
    'rf__min_samples_split': [2, 3, 4, 5],
    'rf__min_samples_leaf': [3, 4, 5, 6, 7, 8, 9]
}

grid_pipeline = GridSearchCV(pipeline,
                             cv = 3,
                             param_grid = params)

In [34]:
grid_pipeline.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('pre',
                                        ColumnTransformer(transformers=[('onehot',
                                                                         Pipeline(steps=[('onehot',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['col9']),
                                                                        ('scaler',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer()),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                               

In [36]:
grid_pipeline.best_params_

{'rf__max_depth': 5, 'rf__min_samples_leaf': 6, 'rf__min_samples_split': 5}

In [38]:
grid_pipeline.best_score_

0.527571633749123

In [41]:
rf_best = grid_pipeline.best_estimator_
rf_best.fit(X_train, y_train)  

Pipeline(steps=[('pre',
                 ColumnTransformer(transformers=[('onehot',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['col9']),
                                                 ('scaler',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['col1', 'col2', 'col3',
                                                   'col4', 'col5', 'col6',
                                                   'col7', 'col8'])])),
                ('selector', SelectFromModel(estimator=Log

In [43]:
rf_best.score(X_test, y_test)

0.5196666666666667

In [ ]:
########## spróbujmy jeszcze na delikatnie zmodyfikowanym zbiorze danych ###########

In [89]:
df2 = pd.read_csv("generated4.csv")
df = df.loc[:, ~df.columns.duplicated()]

## y2 = df2.Class
X2 = df2.drop(['Unnamed: 0', 'Class'], axis = 1)
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size = 0.3)

In [56]:
y2_train

662     1
7544    0
6360    1
7732    1
7448    1
       ..
2280    1
3286    0
5166    0
12      1
9904    1
Name: Class, Length: 7000, dtype: int64

In [92]:
numerical_features2 = list(X2.dtypes[(X2.dtypes != 'object') & (X.dtypes != 'category')].index)

In [93]:
numerical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy = 'mean')),
    ('scaler', StandardScaler())
])

preprocessor2 = ColumnTransformer([
    ('scaler', numerical_transformer, numerical_features2)
])

In [94]:
pipeline2 = Pipeline([
    ('pre', preprocessor2),
    ('selector', SelectFromModel(LogisticRegression())),
    ('rf', RandomForestClassifier())
])

In [95]:
pipeline2.fit(X2_train, y2_train)

Pipeline(steps=[('pre',
                 ColumnTransformer(transformers=[('scaler',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['col1', 'col2', 'col3',
                                                   'col4', 'col5', 'col6',
                                                   'col7', 'col8'])])),
                ('selector', SelectFromModel(estimator=LogisticRegression())),
                ('rf', RandomForestClassifier())])

In [97]:
pipeline2.score(X2_test, y2_test)

0.9286666666666666

In [ ]:
####### teraz gridsearch #########

In [98]:
params = {
    
    'rf__max_depth': [5, 6, 7, 8, 9, 10, 11, 12],
    'rf__min_samples_split': [2, 3, 4, 5],
    'rf__min_samples_leaf': [3, 4, 5, 6, 7, 8, 9]
}

grid_pipeline2 = GridSearchCV(pipeline2,
                             cv = 3,
                             param_grid = params)

In [99]:
grid_pipeline2.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('pre',
                                        ColumnTransformer(transformers=[('scaler',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer()),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['col1',
                                                                          'col2',
                                                                          'col3',
                                                                          'col4',
                                                                          'col5',
                     

In [100]:
grid_pipeline2.best_score_

0.9317134667175493

In [71]:
estimator = grid_pipeline2.best_estimator_

In [72]:
estimator.fit(X_train, y_train)

Pipeline(steps=[('pre',
                 ColumnTransformer(transformers=[('scaler',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['col1', 'col2', 'col3',
                                                   'col4', 'col5', 'col6',
                                                   'col7', 'col8'])])),
                ('selector', SelectFromModel(estimator=LogisticRegression())),
                ('rf',
                 RandomForestClassifier(max_depth=11, min_samples_leaf=6))])

In [74]:
estimator.predict(X_test)

array([0, 1, 1, ..., 1, 1, 1], dtype=int64)

In [73]:
from sklearn.metrics import roc_auc_score


In [75]:
roc_auc_score(estimator.predict(X_test), y_test)

0.9316112531969309

In [ ]:
######## Sprawdzamy co dokładnie z danymi wejsiowymi zrobił preprocessor2 #############

In [76]:
preprocessor2.fit(X2_train)

ColumnTransformer(transformers=[('scaler',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 ['col1', 'col2', 'col3', 'col4', 'col5',
                                  'col6', 'col7', 'col8'])])

In [81]:
x_preproc_train = preprocessor2.transform(X2_train)

In [87]:
pd.DataFrame(x_preproc_train)

,0,1,2,3,4,5,6,7
0,-0.473452,-0.782651,1.667373,-0.473452,0.498506,1.667373,-1.187835,-0.782651
1,0.606351,-0.348236,-0.872544,0.606351,-0.328083,-0.872544,0.728721,-0.348236
2,-0.056521,0.334084,1.595220,-0.056521,-1.494359,1.595220,-0.947132,0.334084
3,0.361425,0.719443,0.304830,0.361425,1.494581,0.304830,-0.083665,0.719443
4,0.295745,0.942021,-0.451524,0.295745,0.820016,-0.451524,0.607729,0.942021
...,...,...,...,...,...,...,...,...
6995,1.421446,0.372390,1.528255,1.421446,0.597713,1.528255,1.368364,0.372390
6996,-0.511030,0.158119,-0.822003,-0.511030,0.182532,-0.822003,1.062433,0.158119
6997,0.373099,0.778840,-0.967925,0.373099,0.495737,-0.967925,0.971164,0.778840
6998,-0.785680,-0.401369,0.081948,-0.785680,1.565270,0.081948,-0.613676,-0.401369


In [101]:
from sklearn.svm import SVC

In [104]:
pipeline_svm = Pipeline([
    ('pre', preprocessor2),
    ('selector', SelectFromModel(LogisticRegression())),
    ('svm', SVC(probability = True))
])

In [105]:
svm_grid = GridSearchCV(pipeline_svm,   
                        {'svm__C': [0.001, 0.01, 0.1, 10,  100]},
                         refit = True,
                         cv = 3,
                        )

In [107]:
svm_grid.fit(X2_train, y2_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('pre',
                                        ColumnTransformer(transformers=[('scaler',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer()),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['col1',
                                                                          'col2',
                                                                          'col3',
                                                                          'col4',
                                                                          'col5',
                     

In [108]:
svm_grid.best_score_

0.9345715687379994